<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Toony_Yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Toony Yourself**

implementation: http://cedro3.com/ai/toonify-yourself/#comment-869

original: 

In [1]:
#StyleGANをGit clone
%tensorflow_version 1.x
!git clone https://github.com/justinpinkney/stylegan2

%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run
 
# フォルダーの作成
!mkdir aligned
!mkdir generated

TensorFlow 1.x selected.
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/stylegan2
CPU says hello.
cudaErrorNoDevice: no CUDA-capable device is detected
mkdir: cannot create directory ‘aligned’: File exists
mkdir: cannot create directory ‘generated’: File exists


In [2]:
# ffhqモデル(Baseモデル)とblendedモデルのダウンロード
#import pretrained_networks
import gdown

 
# use my copy of the blended model to save Doron's download bandwidth
# get the original here https://mega.nz/folder/OtllzJwa#C947mCCdEfMCRTWnDcs4qw
#blended_url = "https://drive.google.com/uc?id=1H73TfV5gQ9ot7slSed_l-lim9X7pMRiU"
blended_url = "https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59" 
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

#エラーがでてうまくいかないので、ダイレクトにダウンロードすることにする 
#_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
#_, _, Gs = pretrained_networks.load_networks(ffhq_url)
gdown.download(blended_url, quiet=False)
gdown.download(ffhq_url, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59
To: /content/stylegan2/ffhq-cartoon-blended-64.pkl
100%|██████████| 382M/382M [00:01<00:00, 199MB/s]
Downloading...
From: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
To: /content/stylegan2/stylegan2-ffhq-config-f.pkl
100%|██████████| 382M/382M [00:01<00:00, 196MB/s]


'stylegan2-ffhq-config-f.pkl'

#**必要な関数を定義**

In [3]:
# 画像表示
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = glob.glob(folder)
    files.sort()
    images=[]
    for i in range(len(files)):
        img = Image.open(files[i])    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()  

# 潜在変数(latents)から生成した画像を表示
import PIL.Image
def display(latents):
    
    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  
    fig = plt.figure(figsize=(30, 40))   
    for i in range(len(latents)):
        vec = latents[i].reshape(1,18,512)
        images =  Gs_blended.components.synthesis.run(vec, randomize_noise=False, **synthesis_kwargs)  
        images = images.transpose((0,2,3,1))     
        PIL.Image.fromarray(images[0], 'RGB')   
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()

# 潜在変数(latents)の順番を指定して、トランジションのGIFを作成する
import os

def generate_gif(latents, idx):

    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)

    image_gif = []
    os.makedirs('my/gif', exist_ok=True)
    for j in range(len(idx)-1):
        for i in range(20):
            latent = latents[idx[j]]+(latents[idx[j+1]]-latents[idx[j]])*i/19
            latent = latent.reshape(1, 18, 512)
            images =  Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs) 
            images = images.transpose((0,2,3,1)) 
            image_one = PIL.Image.fromarray(images[0], 'RGB')
            image_gif.append(image_one.resize((256,256))) 

    image_gif[0].save('./my/gif/anime.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0) 

In [4]:
# サンプル写真のダウンロードと整形
import os 
import bz2
import shutil
import requests
import urllib.request


# rawフォルダを作成
try: 
    shutil.rmtree("/content/stylegan2/raw")
    print("directory presents")
    print("")
except:
    pass
os.makedirs("/content/stylegan2/raw", exist_ok=True)


#Webから画像をダウンロードしてrawフォルダに格納
url = "https://images.unsplash.com/photo-1597223557154-721c1cecc4b0?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8aHVtYW4lMjBmYWNlfGVufDB8fDB8fA%3D%3D&w=1000&q=80"
file_name = "/content/stylegan2/raw/sample.jpg"

response = requests.get(url)
image = response.content
with open(file_name, "wb") as aaa:
    aaa.write(image)

#styleGANのモジュールを用いて画像のアラインメントを行う
!python align_images.py raw aligned

directory presents



In [5]:
# 顔画像の潜在変数を求める 
!python project_images.py --num-steps 500 aligned generated
display_pic('./generated/*.png')

Loading networks from "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"...
2022-06-25 17:08:10.646401: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Failed!
Traceback (most recent call last):
  File "project_images.py", line 122, in <module>
    main()
  File "project_images.py", line 100, in main
    _G, _D, Gs = pretrained_networks.load_networks(args.network_pkl)
  File "/content/stylegan2/pretrained_networks.py", line 76, in load_networks
    G, D, Gs = pickle.load(stream, encoding='latin1')
  File "/content/stylegan2/dnnlib/tflib/network.py", line 297, in __setstate__
    self._init_graph()
  File "/content/stylegan2/dnnlib/tflib/network.py", line 154, in _init_graph
    out_expr = self._build_func(*self.input_templates, **build_kwargs)
  File "<string>", line 491, in G_synthesis_stylegan2
  Fi

<Figure size 2160x2880 with 0 Axes>